<a href="https://colab.research.google.com/github/sravs1407/computational_statistics/blob/main/Anova.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
from scipy import  stats
import scipy.stats

In [ ]:
d=pd.read_excel("/content/mlr.xlsx")

In [ ]:
p=pd.DataFrame(d)
p

,x1,x2,y
0,-5,5,11
1,-4,4,11
2,-1,1,8
3,2,-3,2
4,2,-2,5
5,3,-2,5
6,3,-3,4


In [ ]:
import numpy as np
Y=np.matrix(p['y']).reshape(len(p['y']),1)
print(Y)
p['c']=1
print(p)

[[11]
 [11]
 [ 8]
 [ 2]
 [ 5]
 [ 5]
 [ 4]]
   x1  x2   y  c
0  -5   5  11  1
1  -4   4  11  1
2  -1   1   8  1
3   2  -3   2  1
4   2  -2   5  1
5   3  -2   5  1
6   3  -3   4  1
0   -5
1   -4
2   -1
3    2
4    2
5    3
6    3
Name: x1, dtype: int64


In [ ]:
Xt=np.matrix([p['c'],p['x1'],p['x2']])
print(Xt)
X=np.transpose(Xt)
print(X)
xtx=np.matmul(Xt,X)
print(xtx)
inv=np.linalg.inv(xtx)
print(inv)
xty=np.matmul(Xt,Y)
print(xty)
B=np.matmul(inv,xty)
print(B)
print(B[0,0])

[[ 1  1  1  1  1  1  1]
 [-5 -4 -1  2  2  3  3]
 [ 5  4  1 -3 -2 -2 -3]]
[[ 1 -5  5]
 [ 1 -4  4]
 [ 1 -1  1]
 [ 1  2 -3]
 [ 1  2 -2]
 [ 1  3 -2]
 [ 1  3 -3]]
[[  7   0   0]
 [  0  68 -67]
 [  0 -67  68]]
[[0.14285714 0.         0.        ]
 [0.         0.5037037  0.4962963 ]
 [0.         0.4962963  0.5037037 ]]
[[ 46]
 [-66]
 [ 69]]
[[6.57142857]
 [1.        ]
 [2.        ]]
6.571428571428571


In [ ]:
y_fitted=B[0,0]+B[1,0]*p['x1']+B[2,0]*p['x2']
y_fitted=list(y_fitted)
p['y_fit']=y_fitted
error=p['y']-p['y_fit']
p['e']=error
print(p)

   x1  x2   y  c      y_fit         e
0  -5   5  11  1  11.571429 -0.571429
1  -4   4  11  1  10.571429  0.428571
2  -1   1   8  1   7.571429  0.428571
3   2  -3   2  1   2.571429 -0.571429
4   2  -2   5  1   4.571429  0.428571
5   3  -2   5  1   5.571429 -0.571429
6   3  -3   4  1   3.571429  0.428571


In [ ]:
from re import S
sse=list(p['e']*p['e'])
sse=sum(sse)
print(sse)
y=p['y']
ybar=np.mean(p['y'])
print(ybar)
sst=list((y-ybar)**2)
sst=sum(sst)
print(sst)
ssr=sst-sse
print(ssr)
rsquare=ssr/sst
print(rsquare)
if(rsquare>0.90):
  print("accept the model")
else:
  print("reject the model")

1.7142857142857142
6.571428571428571
73.71428571428571
72.0
0.9767441860465117
accept the model


In [ ]:
#using anova
df1=len(B)-1
df2=len(y)-len(B)
df3=df1+df2
mssr=ssr/df1
msse=sse/df2
f=mssr/msse
anova_table={"source of variation":["regression","error","total"],"sum of sqaures":[ssr,sse,sst],"degrees of freedom":[df1,df2,df3],"mean sum of squares":[mssr,msse," "],"variance ratio":[f,"  ","  "]}
anova_table=pd.DataFrame(anova_table)
print(anova_table)
los=0.05
f_tab=stats.f.ppf(1-los,df1,df2)
print(f_tab)
if(f>f_tab):
  print("reject h0 but accept model based on anova")
else:
  print("accept h0 but reject model based on anova")

  source of variation  sum of sqaures  degrees of freedom mean sum of squares  \
0          regression       72.000000                   2                36.0   
1               error        1.714286                   4            0.428571   
2               total       73.714286                   6                       

  variance ratio  
0           84.0  
1                 
2                 
6.944271909999155
reject h0 but accept model based on anova


In [ ]:
#test of individual parameters
c11=inv[0,0]
print(c11)
c22=inv[1,1]
print(c22)
c33=inv[2,2]
print(c33)
msesquare=sum(p['e']**2)/df2
print(msesquare)
se1=np.sqrt(msesquare*c11)
se2=np.sqrt(msesquare*c22)
se3=np.sqrt(msesquare*c33)
t1=B[0,0]/se1
t2=B[1,0]/se2
t3=B[2,0]/se3
param_table={"predictor":["b0","b1","b2"],"coefficient":[B[0,0],B[1,0],B[2,0]],"standard error":[se1,se2,se3],"t":[t1,t2,t3]}
param_table=pd.DataFrame(param_table)
print(param_table)

0.14285714285714285
0.5037037037037039
0.5037037037037039
0.42857142857142855
  predictor  coefficient  standard error          t
0        b0     6.571429        0.247436  26.558112
1        b1     1.000000        0.464621   2.152290
2        b2     2.000000        0.464621   4.304580


In [ ]:
t_tab=scipy.stats.t.ppf(1-los/2,df2)
print(t_tab)
if(abs(t1)>t_tab):
  print("reject h0 and b0 is contributing to the model")
else:
  print("accept h0 and b0 is not contributing to the model")
if(abs(t2)>t_tab):
  print("reject h0 and b1 is contributing to the model")
else:
  print("accept h0 and b1 is not contributing to the model")
if(abs(t3)>t_tab):
  print("reject h0 and b2 is contributing to the model")
else:
  print("accept h0 and b2 is not contributing to the model")

2.7764451051977987
reject h0 and b0 is contributing to the model
accept h0 and b1 is not contributing to the model
reject h0 and b2 is contributing to the model
